## Assignment 3.2. Image Caption Generation

## Task 2.1: Encoder Decoder Model

Build an image caption generator model, as described in Vinyals, Oriol, et al. "Show and tell: A neural image caption generator." Proceedings of the IEEE conference on computer vision and pattern recognition. 2015. The model shall consist of:

- Image encoder (image feature extractor)
- Caption generator (RNN-based)



In [0]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, GRU, Embedding, TimeDistributed, Dense, RepeatVector, Add, Lambda,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import keras.backend as K

Using TensorFlow backend.


In [0]:
abs_path = '/content/drive/My Drive/Recommender Systems/Assignment 3 - Sequential networks/'
files_path = abs_path + 'Assignment_3_Notebooks_submission/pickel_files/'

In [0]:
# train_features (images)
# train_descriptions (text) ==> We need to upload (train_descriptions, dev_descriptions, test_descriptions) as Pickel files!!!!!
# wordtoix as Pickel file!!!
# num_photos_per_batch computed later
# embedded_matrix as Pickel file!!!!

## Upload Data


In [0]:
import _pickle as cPickle

def read_pickle(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = cPickle.load(f)
    f.close()

    return read_file
  

def read_npy(data_path, file_name):

    f = open(os.path.join(data_path, file_name), 'rb')
    read_file = np.load(f)
    f.close()

    return read_file

# Import Images Pickel Files
train_features = read_pickle(files_path, 'encoded_train_images.pkl')
dev_features = read_pickle(files_path, 'encoded_dev_images.pkl')
test_features = read_pickle(files_path, 'encoded_test_images.pkl')
 
# Import Caption Files (descriptions)
train_descriptions = read_pickle(files_path, 'train_descriptions.pkl')
dev_descriptions = read_pickle(files_path, 'dev_descriptions.pkl')
test_descriptions = read_pickle(files_path, 'test_descriptions.pkl')

# Import Wordtoix dictionary
wordtoix = read_pickle(files_path, 'wordtoix.pkl')

# Import Embedded_matrix
embedding_matrix = read_npy(files_path, 'embedding_matrix.npy')



FileNotFoundError: ignored

## Model

In [0]:
K.clear_session()

In [0]:
# hardcoded variables
rnn_dim = 300; embedding_dim = 300
vocab_size = 1652
decoder_length = 34
max_length = 34
# embedding_matrix = np.zeros((vocab_size, embedding_dim))

### Image Encoder

In [0]:
# image input
image_in = Input(shape=(2048,), name='image_inputs')

In [0]:
# image encoder
fe1 = Dropout(0.5, name='dropout_img_feats')(image_in)
image_dense = Dense(rnn_dim, activation='relu', name = 'dense_img_feats')
fe2 = image_dense(fe1) # reduce the dimension with FC projection

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Caption Generator

In [0]:
# caption input
cap_in = Input(shape=(None,),name='caption_inputs') 

# caption embedding representation (word-based embedding)
embed_cap = Embedding(vocab_size, embedding_dim, weights = [embedding_matrix], trainable = False)
embed_cap_out = embed_cap(cap_in)
drop_cap = Dropout(0.5)
se2 = drop_cap(embed_cap_out)


## encoder
lstm_encoder = LSTM(rnn_dim,  name='lstm_encoder')
# lstm_encoder = GRU(rnn_dim, return_state=True, name='lstm_encoder')
se3 = lstm_encoder(se2)


###### should we include this???
# decoder_inputs = Input(shape=(None,), name='decoder_inputs')
# decoder_embedding = Embedding(NL_VOCAB_SIZE, NL_EMBEDDING_DIM, trainable = True, 
#               weights=[embedding_nl], name='embedding_decoder')
# embedding_output = decoder_embedding(decoder_inputs)
##########

# state input for each decoder time step
s0 = Input(shape=(rnn_dim,), name='s0') # with a dimension of (, rnn_dim)
s = [s0]


# layers initialization
# LSTM/GRU decoder as caption generator
decoder = GRU(rnn_dim, return_state=True)
decoder_dense = Dense(rnn_dim, activation='relu')


# Prediction layer with softmax activation
pred_layer = Dense(vocab_size, activation='softmax')

In [0]:
# print(pred_layer)

In [0]:
probs = []

# process the training per time step (following the max length of captions)
for t in range(decoder_length):
    
    ### YOUR CODE HERE
    x_dec = Lambda(lambda x: x[:,t,:], name='dec_embedding-%s'%t)(se2)
    x_dec = Reshape((1, embedding_dim))(x_dec)
    
    ### ... HOW DO YOU REPRESENT JOINT-REPRESENTATION OF IMAGE-CAPTION AS DECODER INPUT? 
    enc_out_reshape = Reshape((1, embedding_dim))(fe2)
    context_concat = add([x_dec, enc_out_reshape])
    #context_concat = concatenate([x_dec, enc_out_reshape],axis=-1)

    
    ### ... HOW DO YOU INITIALIZE THE RNN-BASED DECODER STATE IN TIME STEP=0? 
    
    if t == 0:
      s = se3
        
    ### ... WHAT IS THE INPUT OF THE DECODER? 
    
    
    s, _ = decoder(context_concat, initial_state = s)
    
    # softmax probability output
    prob = pred_layer(s)
    
    probs.append(prob)
    s = [s]

In [0]:
# # caption input
# cap_in = ### YOUR CODE HERE

# # caption embedding representation (word-based embedding)
# embed_cap = ### YOUR CODE HERE

# # state input for each decoder time step
# s0 = Input(shape=(rnn_dim,), name='s0') # with a dimension of (, rnn_dim)
# s = [s0]

# # LSTM/GRU decoder as caption generator
# decoder = ### YOUR CODE HERE

# # Prediction layer with softmax activation
# pred_layer = Dense(vocab_size, activation='softmax')

In [0]:
# probs = []

# # process the training per time step (following the max length of captions)
# for t in range(decoder_length):
    
#     ### YOUR CODE HERE
    
#     ### ... HOW DO YOU REPRESENT JOINT-REPRESENTATION OF IMAGE-CAPTION AS DECODER INPUT? 
    
#     ### ... HOW DO YOU INITIALIZE THE RNN-BASED DECODER STATE IN TIME STEP=0? 
    
#     if t == 0:
        
#     ### ... WHAT IS THE INPUT OF THE DECODER? 
    
    
#     s, _ = ### YOUR CODE HERE
    
#     # softmax probability output
#     prob = pred_layer(s)
    
#     probs.append(prob)
#     s = [s]

### The model shall be constructed based on the following inputs

In [0]:
# Construct the model
model = Model(inputs=[image_in, cap_in, s0], outputs=probs)
# Compile & run training
adam = optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy')
model.summary()

### Training

In [0]:
### YOUR CODE HERE

In [0]:
# train_descriptions, train_features, wordtoidx, max_length, number_pics_per_batch

In [0]:
# # data generator, intended to be used in a call to model.fit_generator()
# def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
  
#     X1, X2, y = list(), list(), list()
#     n=0
#     # loop for ever over images
#     while 1:
#         for key, desc_list in descriptions.items():
#             n+=1
#             # retrieve the photo feature
#             photo = photos[key+'.jpg']
#             for desc in desc_list:
#                 # encode the sequence
#                 seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
#                 # split one sequence into multiple X, y pairs
#                 for i in range(1, len(seq)):
#                     # split into input and output pair
#                     in_seq, out_seq = seq[:i], seq[i]
#                     # pad input sequence
#                     in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
#                     # encode output sequence
#                     out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
#                     # store
#                     X1.append(photo)
#                     X2.append(in_seq)
#                     y.append(out_seq)
#             # yield the batch data
#             if n==num_photos_per_batch:
#                 yield [[array(X1), array(X2)], array(y)]
#                 X1, X2, y = list(), list(), list()
#                 n=0

In [0]:
# epochs = 10
# number_pics_per_batch = 3
# train_steps = len(train_descriptions)//number_pics_per_batch
# dev_steps = len(dev_descriptions)//number_pics_per_batch
# Callback!!!


# for i in range(epochs):
#     train_generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_batch)
#     dev_generator = data_generator(dev_descriptions, dev_features, wordtoix, max_length, number_pics_per_batch)
#     model.fit_generator(generator, epochs=1, steps_per_epoch=train_steps, verbose=1, callbacks=None, validation_data=dev_generator, validation_steps=dev_steps)
#     model.save('./model_' + str(i) + '.h5')

## Task 2.2: Decoder Model

Based on the completed encoder-decoder, build a decoder model for generating captions using two approaches:
- Greedy search
- Beam search

In [0]:
### YOUR CODE HERE

### Greedy search

In [0]:
### YOUR CODE HERE

### Beam search

In [0]:
### YOUR CODE HERE